In [1]:
# test vllm

from vllm import LLM
import torch

INFO 05-23 03:49:42 [__init__.py:239] Automatically detected platform cuda.


In [2]:


# torch.cuda.empty_cache()

model_name = "/root/autodl-fs/models/TheBloke/Mixtral-8x7B-Instruct-v0___1-GPTQ/"

llm = LLM(model_name, enforce_eager=True, quantization="gptq", dtype=torch.float16)



WARNING 05-23 03:49:47 [config.py:2972] Casting torch.bfloat16 to torch.float16.


INFO 05-23 03:49:55 [config.py:717] This model supports multiple tasks: {'generate', 'embed', 'reward', 'score', 'classify'}. Defaulting to 'generate'.
INFO 05-23 03:49:57 [gptq_marlin.py:147] Detected that the model can run with gptq_marlin, however you specified quantization=gptq explicitly, so forcing gptq. Use quantization=gptq_marlin for faster inference
INFO 05-23 03:49:57 [gptq_bitblas.py:168] Detected that the model can run with gptq_bitblas, however you specified quantization=gptq explicitly, so forcing gptq. Use quantization=gptq_bitblas for faster inference
WARNING 05-23 03:49:57 [config.py:830] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 05-23 03:49:57 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 05-23 03:49:57 [cuda.py:93] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 05-23 03:49:5

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:02<00:00,  2.89s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:02<00:00,  2.89s/it]



INFO 05-23 03:50:09 [loader.py:458] Loading weights took 3.80 seconds
INFO 05-23 03:50:10 [gpu_model_runner.py:1347] Model loading took 22.1664 GiB and 5.911028 seconds
INFO 05-23 03:50:15 [kv_cache_utils.py:634] GPU KV cache size: 38,048 tokens
INFO 05-23 03:50:15 [kv_cache_utils.py:637] Maximum concurrency for 32,768 tokens per request: 1.16x
INFO 05-23 03:50:16 [core.py:159] init engine (profile, create kv cache, warmup model) took 5.56 seconds
INFO 05-23 03:50:16 [core_client.py:439] Core engine process 0 ready.


In [3]:
from vllm import SamplingParams
from coh.utils import parse_predict_answer

# Define the sampling parameters
sampling_params = SamplingParams(
    max_tokens=8000,    # Set the maximum number of tokens to generate
    top_p=1.0,          # Set top-p sampling to 1.0 (equivalent to greedy decoding if temperature is 0)
    temperature=0.0     # Set temperature to 0.0 for deterministic (greedy) sampling
)

from coh.prompts import HIS_TO_ANSWER

outputs = llm.generate(HIS_TO_ANSWER.format(
    histories="""
Government_(Nigeria), Engage_in_diplomatic_cooperation with, Independent_Corrupt_Practices_Commission, on the 339th day; Independent_Corrupt_Practices_Commission, Arrest_or_detain_or_charge_with_legal_action to, Citizen_(Nigeria), on the 308th day;
Government_(Nigeria), Criticize_or_denounce, Boko_Haram, on the 337th day; 
Boko_Haram, Use_conventional_military_force to, Citizen_(Nigeria), on the 336th day;
Government_(Nigeria), Threaten, Education_(Nigeria), on the 337th day; 
Education_(Nigeria), Make_statement to, Muslim_(Nigeria), on the 332nd day;
Government_(Nigeria), Make_optimistic_comment on, Citizen_(Nigeria), on the 336th day;
Citizen_(Nigeria), Make_an_appeal_or_request to, Member_of_the_Judiciary_(Nigeria), on the 331st day;
· · · · · ·
    """,
    query="""
Government_(Nigeria), Make_an_appeal_or_request to, whom, on the 340th day?
    """
), sampling_params=sampling_params)


for output in outputs:
    # The original prompt that was sent
    original_prompt = output.prompt
    print(f"Original Prompt: {original_prompt}")

    # Access the generated completions
    for completion in output.outputs:
        generated_text = completion.text
        finish_reason = completion.finish_reason
        # You can also access completion.token_ids, completion.cumulative_logprob, etc.
        print(f"Generated Text: {generated_text}")
        print(parse_predict_answer(generated_text))
        print(f"Finish Reason: {finish_reason}")

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Original Prompt: 
You must be able to correctly predict the {whom} of the given query from a given text consisting
of multiple historical events in the form of “{subject} {relation} {object} {time}” and the query in
the form of “{subject} {relation} {whom} {time}?” You must output several {object} that you
think may be the answer to the given query based on the given historical events. Please list all
possible {object} which may be answers to the query. Please assign each answer a serial number
to represent its probability of being the correct answer. Note that answers with a high probability
of being correct should be listed first.
Here are the given historical events:

Government_(Nigeria), Engage_in_diplomatic_cooperation with, Independent_Corrupt_Practices_Commission, on the 339th day; Independent_Corrupt_Practices_Commission, Arrest_or_detain_or_charge_with_legal_action to, Citizen_(Nigeria), on the 308th day;
Government_(Nigeria), Criticize_or_denounce, Boko_Haram, on the 337th d